In [4]:
# Install liblinear: sudo apt-get install python-liblinear
# from liblinearutil import *
import numpy as np

In [6]:
def read_problem(file_name):
    y = []
    x = []
    feature_len = 22
    for line in open(file_name):
        tmp = line.split(' ',1)
        y = y + [int(tmp[0])]
        vec = np.zeros(feature_len)
        for each in tmp[1].split():
            index,val = each.split(':')
            vec[int(index)-1] = float(val)
        x.append(vec)
    y = np.array(y)
    x = np.array(x)
    return y,x

# Gradient ascent step for dual variables

In [85]:
def get_tp_tn(x,y,w):
    h = x.dot(w)
    ind = np.where(h>0)
    y1 = -1*np.ones_like(y)
    y1[ind] = 1
    tp = sum(np.multiply(1+y,1+y1)/4)
    tn = sum(np.multiply(1-y,1-y1)/4)
    return tp,tn

def get_projection(alpha,beta):
    if alpha>0 and beta>0 and alpha*beta>=1.0/4:
        return alpha,beta
    a = 16
    b = -16*beta
    c = 0
    d = 4*alpha
    e = -1
    D0 = c*c - 3*b*d + 12*a*e
    D1 = 2*c*c*c-9*b*c*d+27*b*b*e+27*a*d*d-72*a*c*e
    p = (8*a*c-3*b*b)/(8*a*a)
    q = (b*b*b-4*a*b*c+8*a*a*d)
    Q = np.power((D1+np.sqrt(D1*D1-4*D0*D0*D0))/2.0,1/3)
    s = 0.5*np.sqrt(-2/3*p+1/(3*a)*(Q+D0/Q))
    y1 = -float(b)/(4*a) - s + 0.5*np.sqrt(-4*s*s-2*p+q/s)
    y2 = -float(b)/(4*a) - s - 0.5*np.sqrt(-4*s*s-2*p+q/s)
    y3 = -float(b)/(4*a) + s + 0.5*np.sqrt(-4*s*s-2*p+q/s)
    y4 = -float(b)/(4*a) + s - 0.5*np.sqrt(-4*s*s-2*p+q/s)
    x1 = 1/(4*y1)
    if x1>=1/4 and y1>=1/4:
        return x1,y1
    x2 = 1/(4*y2)
    if x2>=1/4 and y2>=1/4:
        return x2,y2
    x3 = 1/(4*y3)
    if x3>=1/4 and y3>=1/4:
        return x3,y3
    x4 = 1/(4*y4)
    if x4>=1/4 and y4>=1/4:
        return x4,y4
    return 1,1

def gradient_ascent(x,y,w,a,b,alpha=0.1,maxiter=50):
    n = x.shape[0]
    n_pos = len([i for i in range(n) if y[i]==1])
    n_neg = n-n_pos
    ind_pos = [i for i in range(n) if y[i]>0]
    ind_neg = [i for i in range(n) if y[i]<0]
    x_pos = x[ind_pos]
    x_neg = x[ind_neg]
    tpr,tnr = get_tp_tn(x,y,w)
    tpr /= float(n_pos)
    tnr /= float(n_neg)
    epsilon = 0.0001
    print('TPR =',tpr,'TNR =',tnr)
    for it in range(maxiter):
#         alpha = 1.0/(it+1)
        # for positives
        tmp = 1 - (2*n_pos)/(a*n)*x_pos.dot(w)
        j = np.where(tmp>0)[0]
        tmp = np.zeros_like(tmp)
        tmp[j]=1.0/n_pos
        ga = sum(tmp)
        a += alpha*ga
#         print('Missed pos:',len(j))
        # for negatives
        tmp = 1 + (2*n_neg)/(b*n)*x_neg.dot(w)
        j = np.where(tmp>0)[0]
        tmp = np.zeros_like(tmp)
        tmp[j]=1.0/n_neg
        gb = sum(tmp)
        b += alpha*gb
        print(a,b)
#         print('Missed neg:',len(j))
        # Projection to a,b \in R+ \intersect ab>1/4
        a,b = get_projection(a,b)
#         print('ga =',ga,'gb =',gb)
        if ga<epsilon and gb<epsilon:
            break
    return a,b

## Liblinear for primal variable w

In [7]:
# customized liblinear
import sys
sys.path.insert(0, "/home/debojyoti/opt/liblinear-2.1")
from ppython import liblinear
from ppython.liblinear import *
from ppython.liblinearutil import *

In [1]:
def modifyx(x,y,a,b,n,n_pos,n_neg):
    scale_pos = float(2*n_pos)/(a*n)
    scale_neg = float(2*n_neg)/(b*n)
    for i in range(len(y)):
        if y[i]==1:
            x[i].update((key,val*scale_pos) for key,val in x[i].items())
        else:
            x[i].update((key,val*scale_neg) for key,val in x[i].items())
    return x
# init section
a,b = 0.5,0.5
y_lst,x_lst = read_problem('./data/ijcnn1.tr') # x as numpy list
n_dim = len(x_lst[0])
w = np.random.uniform(low=-1,high=1,size=(n_dim,))
# w = np.zeros(n_dim)
y_orig,x_orig = svm_read_problem('./data/ijcnn1.tr') # x in liblinear compatible dictionary format
n = len(y_lst)
n_pos = len([i for i in range(n) if y_lst[i]==1])
n_neg = n-n_pos
c = 5.0
# Iterative section: Gradient Ascent & Liblinear
for i in range(10):
    a,b = gradient_ascent(x_lst,y_lst,w,a,b) #Gradient ascent
    c1 = float(a)/n_pos
    c2 = float(b)/n_neg
    param = parameter('-s 3 -w1 {} -w-1 {} -c {}'.format(c1,c2,c))
    x = modifyx(x_orig,y_orig,a,b,n,n_pos,n_neg)
    prob = problem(y_orig,x)
    model = train(prob,param)
    # Get model parameters
    w = model.get_decfun()[0]
    print('a =',a,'b =',b)